# More on Embeddings 

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
import numpy as np
import cufflinks as cf; cf.go_offline()

In [3]:
wine_reviews = pd.read_csv("../Final project/data/winemag-data-130k-v2.csv")
wine_reviews.shape

(129971, 14)

In [4]:
wine_reviews.sample(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
84205,84205,Italy,"Made entirely with native Gaglioppo grapes, th...",NaN,89,12.0,Southern Italy,Cirò,NaN,Kerin O’Keefe,@kerinokeefe,Librandi 2014 Rosato (Cirò),Rosato,Librandi
106634,106634,US,"This runs to the tart, lemony style of Pinot G...",NaN,87,12.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Acrobat 2011 Pinot Gris (Oregon),Pinot Gris,Acrobat
5803,5803,US,"This is an oddly perfumed and puckering wine, ...",NaN,83,50.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Housley's Century Oak 2013 Cabernet Sauvignon ...,Cabernet Sauvignon,Housley's Century Oak
11459,11459,US,"A full-bodied Pinot Noir, this bears ripe cher...",NaN,87,12.0,California,Central Coast,Central Coast,NaN,NaN,Ballard Lane 2010 Pinot Noir (Central Coast),Pinot Noir,Ballard Lane
123362,123362,Austria,"Fragrant, fresh, young and zippy, this is a si...",Zweigelt,89,12.0,Niederösterreich,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Winzer Krems 2014 Zweigelt Rosé (Niederösterre...,Rosé,Winzer Krems


In [5]:
text = "There's no shortage of spice in this value"
words = text.split(' ')
words

["There's", 'no', 'shortage', 'of', 'spice', 'in', 'this', 'value']

In [26]:
def create_x_hot_encoding(vocab_len, dim, emb_size):
    t = np.zeros(vocab_len * emb_size)
    t[emb_size * dim: emb_size*dim + emb_size] = 1
    return t


In [31]:
create_x_hot_encoding(10, 1, 3)

array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Preparing the data

In [14]:
train_data = wine_reviews.sample(frac = 0.8)
test_data = wine_reviews[~wine_reviews.index.isin(train_data.index)]
assert(len(train_data) + len(test_data) == len(wine_reviews))

In [15]:
len(test_data), len(train_data)

(25994, 103977)

#### Tokenization

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer